[source](https://www.mathworks.com/help/optim/ug/sudoku-puzzles-problem-based.html)

In [1]:
import numpy as np
import scipy as sp
import time
from codetiming import Timer


In [2]:
#The sudocu from the problem
empty_sudoku = np.array(
   [[0, 0, 1, 0, 0, 0, 0, 0, 2],
    [0, 0, 0, 3, 0, 4, 5, 0, 0],
    [6, 0, 0, 7, 0, 0, 1, 0, 0],
    [0, 4, 0, 5, 0, 0, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 0, 8, 0],
    [0, 0, 0, 0, 0, 6, 0, 9, 0],
    [0, 0, 5, 0, 0, 9, 0, 0, 4],
    [0, 0, 8, 2, 0, 1, 0, 0, 0],
    [3, 0, 0, 0, 0, 0, 7, 0, 0]]
)

#a sample sudocu from the internet
sample_solution = np.array(
    [[7, 1, 3, 5, 2, 4, 6, 9, 8],
     [5, 2, 9, 6, 1, 8, 3, 4, 7],
     [6, 4, 8, 7, 3, 9,	2, 5, 1],
     [1, 5, 2, 9, 4, 7, 8, 3, 6],
     [8, 3, 6, 1, 5, 2, 9, 7, 4],
     [4, 9, 7, 3, 8, 6, 5, 1, 2],
     [3, 8, 5, 4, 6, 1, 7, 2, 9],
     [9, 6,	1, 2, 7, 5, 4, 8, 3],
     [2, 7, 4, 8, 9, 3, 1, 6, 5]]
)

#the same sample sudoku with some elements deleted
empty_sample_solution = np.array(
    [[0, 0, 0, 5, 2, 4, 6, 9, 8],
     [5, 2, 0, 6, 1, 8, 3, 4, 7],
     [6, 4, 8, 7, 3, 9,	2, 5, 1],
     [1, 5, 2, 9, 4, 7, 8, 3, 6],
     [8, 3, 6, 1, 5, 2, 9, 7, 4],
     [4, 9, 7, 3, 8, 6, 5, 1, 2],
     [3, 8, 5, 4, 6, 1, 7, 2, 9],
     [9, 6,	1, 2, 7, 5, 4, 8, 3],
     [2, 7, 4, 8, 9, 3, 1, 6, 5]]
)

#A very empty sudoku
very_empty_clues = np.zeros((9,9), dtype = int)
very_empty_clues[0,1] = 3
very_empty_clues[0,2] = 2
very_empty_clues[0,3] = 1



In [3]:
#you can also just turn the matrix into a sparse one in index format, that might be faster
def sudoku_array_to_clues(clue_array):
    clues_inds_rows, clues_inds_cols = np.where(clue_array>0)
    clues_vals = clue_array[clues_inds_rows, clues_inds_cols]
    return clues_inds_rows, clues_inds_cols, clues_vals

def array_to_vector(sudoku_array):
    #turn a sudoku values array into a binary vector
    flat_values = sudoku_array.flatten()
    binary_values = np.zeros((flat_values.shape[0], 9), dtype = int)
    indices = np.arange(flat_values.shape[0], dtype = int)
    binary_values[indices, flat_values-1] = 1
    return binary_values.flatten()

def vector_to_array(binary_solution):
    #turn a binary vector into a soduku values array
    binary_values = binary_solution.reshape((81,9)) #magig numbers are sudoky dims and number of values possible
    _, flat_values = np.where(binary_values)
    sudoku_array = (flat_values+1).reshape(9,9)
    return sudoku_array

bin_sample_solution = array_to_vector(sample_solution)
vector_to_array(bin_sample_solution)
    

array([[7, 1, 3, 5, 2, 4, 6, 9, 8],
       [5, 2, 9, 6, 1, 8, 3, 4, 7],
       [6, 4, 8, 7, 3, 9, 2, 5, 1],
       [1, 5, 2, 9, 4, 7, 8, 3, 6],
       [8, 3, 6, 1, 5, 2, 9, 7, 4],
       [4, 9, 7, 3, 8, 6, 5, 1, 2],
       [3, 8, 5, 4, 6, 1, 7, 2, 9],
       [9, 6, 1, 2, 7, 5, 4, 8, 3],
       [2, 7, 4, 8, 9, 3, 1, 6, 5]])

In [4]:
#these are operatiors that should apply to a 729 long solution vector x
#x 
NCOLS = 9
NROWS = 9
NVALS = 9

def generate_one_value_constr():
    #[1,1,1,1,1,1,1,1,1, 0,0,0,0,0,....]
    #[0,0,0,0,0,0,0,0,0, 1,1,1,1,1,....
    #...
    return np.kron(np.eye(NROWS*NCOLS), np.ones((1,NVALS))) #np.kron(np.eye(2), np.kron(np.eye(2), np.ones((1,4))))

def generate_row_uniq_constr():
    #[1,..,0, 1,..,0, ... 1,... 0,  0,1,..,0, 0, 
    return np.kron(np.eye(NROWS), np.kron(np.ones((1,NCOLS)), np.eye(NVALS))) #

def generate_col_uniq_constr():
    return np.kron(np.ones((1,NROWS)), np.eye(NCOLS*NVALS))

def generate_block_uniq_constr():
    return np.kron(np.eye(3), np.kron(np.ones((1,3)), np.kron(np.eye(3), np.kron( np.ones((1,3)), np.eye(9)))))

def generate_clues_contr(clues_inds_rows, clues_inds_cols, clues_vals):
    #x[i,j,m] = 1
    op = np.zeros((clues_vals.shape[0], NROWS*NCOLS*NVALS))
    indices = np.arange(clues_vals.shape[0])
    op[indices, NVALS*NCOLS*clues_inds_rows + NVALS*clues_inds_cols + clues_vals-1] = 1
    return op



In [5]:
def setup_ILP(clue_array):
    clues_inds_rows, clues_inds_cols, clues_vals = sudoku_array_to_clues(clue_array)
    
    A_one_val = generate_one_value_constr()
    A_row = generate_row_uniq_constr()
    A_col = generate_col_uniq_constr()
    A_block = generate_block_uniq_constr()
    A_clues = generate_clues_contr(clues_inds_rows, clues_inds_cols, clues_vals)

    A_full = np.vstack([A_one_val, A_row, A_col, A_block, A_clues])
    c = np.linspace(0,1,A_full.shape[1]) #np.zeros(solution_size, dtype = np.float64)
    return c, A_full
    

def solve_sudoku_scipy(c, A_full):
    solution_size = A_full.shape[1]
    integrality = np.ones(solution_size, dtype = int)
    bounds = sp.optimize.Bounds(lb=0, ub=1) #somthing with 0 and 
    constraints = sp.optimize.LinearConstraint(A_full, lb=1, ub=1) #somthing with A
    return sp.optimize.milp( c,integrality = integrality, bounds = bounds, constraints = constraints)




In [6]:
with Timer("Setup 1"):
    c, A_full = setup_ILP(empty_sample_solution)
with Timer("Scipy solver 1"):
    res = solve_sudoku_scipy(c, A_full)

print(Timer.timers)

vector_to_array(res.x)


Elapsed time: 0.0043 seconds
Elapsed time: 0.2057 seconds
{'Setup 1': 0.004348344169557095, 'Scipy solver 1': 0.2056662819813937}


array([[7, 1, 3, 5, 2, 4, 6, 9, 8],
       [5, 2, 9, 6, 1, 8, 3, 4, 7],
       [6, 4, 8, 7, 3, 9, 2, 5, 1],
       [1, 5, 2, 9, 4, 7, 8, 3, 6],
       [8, 3, 6, 1, 5, 2, 9, 7, 4],
       [4, 9, 7, 3, 8, 6, 5, 1, 2],
       [3, 8, 5, 4, 6, 1, 7, 2, 9],
       [9, 6, 1, 2, 7, 5, 4, 8, 3],
       [2, 7, 4, 8, 9, 3, 1, 6, 5]])

In [7]:
with Timer("Setup 2"):
    c, A_full = setup_ILP(empty_sudoku)
with Timer("Scipy solver 2"):
    res = solve_sudoku_scipy(c, A_full)

print(Timer.timers)

vector_to_array(res.x)

Elapsed time: 0.0038 seconds
Elapsed time: 0.0110 seconds
{'Setup 1': 0.004348344169557095, 'Scipy solver 1': 0.2056662819813937, 'Setup 2': 0.0038175201043486595, 'Scipy solver 2': 0.010952536948025227}


array([[7, 3, 1, 8, 6, 5, 9, 4, 2],
       [8, 9, 2, 3, 1, 4, 5, 6, 7],
       [6, 5, 4, 7, 9, 2, 1, 3, 8],
       [9, 4, 6, 5, 8, 3, 2, 7, 1],
       [1, 2, 3, 9, 4, 7, 6, 8, 5],
       [5, 8, 7, 1, 2, 6, 4, 9, 3],
       [2, 7, 5, 6, 3, 9, 8, 1, 4],
       [4, 6, 8, 2, 7, 1, 3, 5, 9],
       [3, 1, 9, 4, 5, 8, 7, 2, 6]])

In [8]:
with Timer("Setup 3"):
    c, A_full = setup_ILP(very_empty_clues)
with Timer("Scipy solver 3"):
    res = solve_sudoku_scipy(c, A_full)

print(Timer.timers)

vector_to_array(res.x)

Elapsed time: 0.0023 seconds
Elapsed time: 0.5283 seconds
{'Setup 1': 0.004348344169557095, 'Scipy solver 1': 0.2056662819813937, 'Setup 2': 0.0038175201043486595, 'Scipy solver 2': 0.010952536948025227, 'Setup 3': 0.0023200740106403828, 'Scipy solver 3': 0.5283199949190021}


array([[6, 3, 2, 1, 7, 5, 9, 4, 8],
       [4, 9, 7, 3, 6, 8, 1, 5, 2],
       [5, 1, 8, 2, 9, 4, 7, 6, 3],
       [8, 4, 9, 7, 5, 1, 2, 3, 6],
       [1, 2, 6, 8, 3, 9, 4, 7, 5],
       [3, 7, 5, 6, 4, 2, 8, 1, 9],
       [7, 5, 1, 9, 2, 6, 3, 8, 4],
       [9, 8, 4, 5, 1, 3, 6, 2, 7],
       [2, 6, 3, 4, 8, 7, 5, 9, 1]])